In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from pandas_profiling import ProfileReport
import graphviz
import csv

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, precision_recall_curve, r2_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
from sklearn.datasets import load_diabetes
from sklearn.datasets import load_breast_cancer

In [5]:
ins_df = pd.read_csv('insurance.csv')
ins_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [8]:
X = pd.get_dummies(ins_df.iloc[:, :-1])
y = ins_df.loc[:, 'charges']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [12]:
pipe = make_pipeline(MinMaxScaler(), LinearRegression()).fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7958786376014415

In [17]:
pipe.named_steps['linearregression'].coef_, pipe.named_steps['linearregression'].intercept_

(array([ 1.15069979e+04,  1.23886402e+04,  2.21106880e+03, -6.63153624e+00,
         6.63153624e+00, -1.18283103e+04,  1.18283103e+04,  5.35344576e+02,
         3.42211643e+02, -5.44457080e+02, -3.33099139e+02]),
 9582.347435907923)

In [27]:
lr_coef = pipe.named_steps['linearregression'].coef_
lr_intercept = pipe.named_steps['linearregression'].intercept_
(pd.DataFrame(lr_coef)
.rename(dict(enumerate(X.columns)))
.apply(lambda df: df.div(df.sum(axis=0)))
)

,0
age,0.440768
bmi,0.474539
children,0.084694
sex_female,-0.000254
sex_male,0.000254
smoker_no,-0.453076
smoker_yes,0.453076
region_northeast,0.020506
region_northwest,0.013108
region_southeast,-0.020855


In [40]:
pipe = make_pipeline(MinMaxScaler(), 
SGDRegressor(loss='squared_error', penalty='l1', alpha=0.1, learning_rate='invscaling', max_iter=1000, random_state=0)).fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7947894067641075

In [41]:
pipe.named_steps['sgdregressor'].coef_, pipe.named_steps['sgdregressor'].intercept_, pipe.named_steps['sgdregressor'].n_iter_

(array([11593.79110741, 10382.67267152,  2205.8369832 ,  2278.09324513,
         2297.09892774, -9525.95351756, 14114.44893084,  1593.16722028,
         1395.66771951,   770.66960293,   802.38438972]),
 array([4588.49541328]),
 85)

In [42]:
lr_coef = pipe.named_steps['sgdregressor'].coef_
lr_intercept = pipe.named_steps['sgdregressor'].intercept_
(pd.DataFrame(lr_coef)
.rename(dict(enumerate(X.columns)))
.apply(lambda df: df.div(df.sum(axis=0)))
)

,0
age,0.305841
bmi,0.273892
children,0.058189
sex_female,0.060096
sex_male,0.060597
smoker_no,-0.251292
smoker_yes,0.372336
region_northeast,0.042027
region_northwest,0.036817
region_southeast,0.020330


In [44]:
pipe = make_pipeline(MinMaxScaler(), PolynomialFeatures(degree=3, interaction_only=True), LinearRegression()).fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.8342006759566594, 0.8664053037323034)

In [45]:
cer_df = pd.read_csv('cereal.csv')
cer_df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


In [50]:
pd.concat([cer_df.nunique(), cer_df.dtypes], axis=1).head()

,0,1
name,77,object
mfr,7,object
type,2,object
calories,11,int64
protein,6,int64


In [53]:
X = pd.get_dummies(cer_df.iloc[:, 1:-1])
y = cer_df.loc[:, 'rating']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [74]:
pipe = make_pipeline(MinMaxScaler(), PolynomialFeatures(degree=3, interaction_only=True), LinearRegression()).fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(1.0, 0.9720882221344329)

In [75]:
pipe.named_steps['linearregression'].coef_, pipe.named_steps['linearregression'].intercept_

(array([ 2.93607069, -3.30030697,  0.63408809, ...,  0.        ,
         0.        ,  0.        ]),
 49.24990667433556)

In [58]:
lr_coef = pipe.named_steps['linearregression'].coef_
lr_intercept = pipe.named_steps['linearregression'].intercept_
(pd.DataFrame(lr_coef)
#.rename(dict(enumerate(X.columns)))
.apply(lambda df: df.div(df.sum(axis=0)))
)

,0
0,-0.299307
1,0.336438
2,-0.064640
3,0.430792
4,0.402182
...,...
1557,-0.000000
1558,-0.000000
1559,-0.000000
1560,-0.000000


In [76]:
pipe = make_pipeline(LinearRegression()).fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.9999999999999997, 0.9999999999999993)

In [77]:
pd.concat([pd.Series(y_test).reset_index(drop=True), pd.Series(pipe.predict(X_test))], axis=1).head()

,rating,0
0,29.924285,29.924285
1,49.787445,49.787445
2,39.703400,39.703400
3,60.756112,60.756112
4,45.811716,45.811716


In [78]:
lr_reg = LinearRegression().fit(X_train, y_train)
lr_reg.score(X_test, y_test)

0.9999999999999993

In [80]:
diabetes = load_diabetes()
X = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])
y = pd.DataFrame(diabetes['target'], columns=['target'])

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [112]:
alpha = np.logspace(-3, 1, 5)
data = []
for i, a in enumerate(alpha):
    ridge=Ridge(alpha=a)
    ridge.fit(X.values, y.values.ravel())
    data.append(pd.Series(np.hstack([ridge.coef_])))
df_ridge = pd.DataFrame(data, index=alpha)
df_ridge.columns = X.columns
df_ridge

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0.001,-9.551414,-239.090354,520.363367,323.828627,-712.328205,413.383794,65.811629,167.513774,720.944468,68.122100
0.010,-7.199457,-234.552930,520.583136,320.523356,-380.607066,150.483752,-78.591232,130.313059,592.349587,71.133768
0.100,1.307349,-207.194814,489.691080,301.769437,-83.466074,-70.828096,-188.680164,115.712703,443.814054,86.748539
1.000,29.465746,-83.154885,306.351627,201.629434,5.909369,-29.515927,-152.040465,117.311715,262.944995,111.878718
10.000,19.812822,-0.918458,75.416167,55.025419,19.924600,13.948686,-47.553816,48.259420,70.144068,44.213876


In [113]:
alpha = np.logspace(-3, 1, 5)
data = []
for i, a in enumerate(alpha):
    lasso=Lasso(alpha=a)
    lasso.fit(X.values, y.values.ravel())
    data.append(pd.Series(np.hstack([lasso.coef_])))
df_lasso = pd.DataFrame(data, index=alpha)
df_lasso.columns = X.columns
df_lasso

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0.001,-8.998449,-238.899740,520.261362,323.429484,-720.251734,421.405141,66.734168,164.448873,725.340440,67.475538
0.010,-1.306575,-228.822331,525.560658,316.175320,-307.013677,89.321688,-105.081398,119.597989,571.330871,65.007316
0.100,-0.000000,-155.362882,517.182017,275.082351,-52.540269,-0.000000,-210.159753,0.000000,483.914409,33.672821
1.000,0.000000,-0.000000,367.701852,6.301904,0.000000,0.000000,-0.000000,0.000000,307.605700,0.000000
10.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000


In [118]:
ela_reg = ElasticNet(alpha=0.001, l1_ratio=0.5).fit(X_train, y_train)
ela_reg.score(X_train, y_train), ela_reg.score(X_test, y_test)

(0.5452462951988923, 0.3751029973603024)

In [119]:
ridge = Ridge(alpha=0.001).fit(X_train, y_train)
ridge.score(X_train, y_train), ridge.score(X_test, y_test)

(0.555407960728878, 0.3586357962100011)

In [121]:
pipe = make_pipeline(MinMaxScaler(), Ridge(alpha=0.001)).fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.5554369442153996, 0.35933536601101934)

In [145]:
pipe = make_pipeline(RandomForestRegressor(n_estimators=100, max_leaf_nodes=12)).fit(X_train, y_train.values.ravel())
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.6855714234795974, 0.2832947833343996)

In [146]:
y_test.assign(predict=pipe.predict(X_test)).head()
# pipe.predict(X_test)

,target,predict
362,321.0,243.469147
249,215.0,251.529293
271,127.0,170.539654
435,64.0,101.772685
400,175.0,191.908346


In [154]:
pipe = make_pipeline(AdaBoostRegressor(base_estimator=Ridge(alpha=0.01))).fit(X_train, y_train.values.ravel())
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

(0.5506359244166652, 0.3564636524275341)

In [173]:
body_df = pd.read_csv('bodyPerformance.csv')
body_df['gender'] = body_df['gender'].replace({'M':1, 'F':0})
body_df['class'] = body_df['class'].replace({'A':0, 'B':1, 'C':2, 'D':3})
body_df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,1,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,2
1,25.0,1,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,0
2,31.0,1,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,2
3,32.0,1,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1
4,28.0,1,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1


In [174]:
X = body_df.iloc[:, :-1]
y = body_df.loc[:, 'class']

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

In [183]:
pipe = make_pipeline(LogisticRegression()).fit(X_train, y_train)
pipe.score(X_train, y_train), pipe.score(X_test, y_test)

c:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.6043409000398248, 0.5954016124216184)

In [199]:
prob = pd.DataFrame(pipe.named_steps['logisticregression'].predict_proba(X_test).round(3), columns=['A확률', 'B확률', 'C확률', 'D확률'])
expect = pd.DataFrame(pipe.predict(X_test), columns=['예상'])
actual = y_test.reset_index(drop=True).to_frame(name='실제')
pd.concat([prob, expect, actual], axis=1)

,A확률,B확률,C확률,D확률,예상,실제
0,0.392,0.426,0.167,0.015,1,0
1,0.000,0.001,0.045,0.954,3,3
2,0.862,0.124,0.014,0.000,0,0
3,0.001,0.054,0.308,0.636,3,3
4,0.012,0.183,0.423,0.382,2,3
...,...,...,...,...,...,...
3344,0.000,0.005,0.142,0.853,3,3
3345,0.007,0.117,0.383,0.493,3,2
3346,0.028,0.245,0.540,0.186,2,1
3347,0.265,0.371,0.350,0.013,1,0
